In [1]:
import os
import re
import json
import pandas as pd
from tqdm import tqdm
from time import sleep

# Prepare datasets

In [2]:
def get_rubq_questions(split='test'):
    if os.path.exists('../data/rubq/rubq_test_qa_pairs.json'):
        with open(f'../data/rubq/rubq_test_qa_pairs.json', 'r') as f:
            return json.load(f)
            
    with open(f'../data/rubq/rubq_test.json', 'r') as f:
        data = json.load(f)
    
    questions = []
    
    for item in tqdm(data):
        questions.append({
                'question': item['question_text'],
                'answer': item['answer_text']
        })

    return questions

def get_qald_questions(split='test'):
    if os.path.exists('../data/qald/qald_test_qa_pairs.json'):
        with open(f'../data/qald/qald_test_qa_pairs.json', 'r') as f:
            return json.load(f)
        
    questions = []

    with open(f'../data/qald/qald_{split}.json', 'r') as f:
        data = json.load(f)['questions']
        
    for item in tqdm(data):
        
        answer_list = []
        for a in item['answers']:
            if a.get('results'):
                for b in a['results']['bindings']:
                    if b.get('resultCnt'):
                        answer = b['resultCnt']['value']
                    else:
                        answer = b['result']['value']
                    
                    match = re.match(r'^https?://www\.wikidata\.org/entity/(Q\d+)$', answer)
                    if match:
                        answer = match.group(1)
                        answer = str(client.get(answer).label)
                    
                    answer_list.append(answer)
        
        for q in item['question']:
            if q['language'] == 'en':
                question = q['string']
                
        questions.append({
            'question': question,
            'answer': answer_list
        })
        
    return questions

def get_lcquad_questions(split='test'):
    if os.path.exists('../data/lcquad/lcquad_test_qa_pairs.json'):
        with open(f'../data/lcquad/lcquad_test_qa_pairs.json', 'r') as f:
            return json.load(f)
            
    questions = []
    
    with open(f'../data/lcquad/lcquad_2_{split}.json', 'r') as f:
        data = json.load(f)
            
    for item in tqdm(data):
        results = None
        for i in range(10):
            try:
                sparql.setQuery(item['query'])
                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
                break
            except:
                sleep(1)
                continue
        
        answer_list = []
        if results:
            if results.get('results'):
                for result in results["results"]["bindings"]:
                    if result.get('answer'):
                        label = result["answer"]["value"]
                    elif result.get('obj'):
                        label = result["obj"]["value"]
                    answer_list.append(label)
            
        questions.append({
            'question': item['en_question'],
            'answer': answer_list,
        })
    
    return questions


def get_pat_questions(split='test'):
    if os.path.exists('../data/pat/custom_pat_test_qa_pairs.json'):
        with open(f'../data/pat/custom_pat_test_qa_pairs.json', 'r') as f:
            return json.load(f)
            
    with open(f'../data/pat/custom_iid_pat_{split}.json', 'r') as f:
        data = json.load(f)
    
    questions = []
    
    for item in data:
        questions.append({
            'question': item['question'],
            'answer': item['text answers']
        })
    
    return questions

def batch_questions(data, batch_size=10):
    questions = []
    dataset = pd.DataFrame(data)
    for start in range(0, len(dataset), batch_size):
        batch = dataset.iloc[start:start + batch_size]
        questions.append('\n'.join([f"{i}) {row.question}" for i, row in batch.iterrows()]))
    return questions

# ChatGPT

In [3]:
from utils import APIClient

# api_key = ...
client = APIClient(api_key=api_key, max_retries=3)

In [26]:
def create_task_list(dataset, batch_size=60, model_name='gpt-4o', lang='en'):
    if lang == 'ru':
        system_prompt = ru_system_prompt
        prompt = 'Список вопросов:\n'
    elif lang == 'en':
        system_prompt = en_system_prompt
        prompt = 'Questions:\n'
    
    task_list = []
    
    for i, batch in enumerate(batch_questions(dataset, batch_size)):
        content = prompt + batch
    
        task_list.append({
            'task_id': i,
            'url': 'https://api.openai.com/v1/chat/completions',
            'body': {
                'model': model_name,
                'messages': [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": content}
                ]
            }
        })
        
    return task_list

def unpack_answer(result):
    result_dict = {}
    
    # Split the data by lines and process each line
    for line in result.splitlines():
        if not 'ответы' in line.lower() and 'answers' not in line.lower():
            try:
                key, value = line.split(") ", 1)  # Split at the first occurrence of ". "
            except:
                print(line)
            result_dict[int(key)] = value.strip()  # Convert key to int and strip any whitespace from value

    return result_dict

def save_answer(completed_tasks, dataset_name):
    result = {}
    for id, complete_task in completed_tasks.items():
        result.update(unpack_answer(complete_task[0]['content']))
    
    with open(f"../data/gpt_results/{dataset_name}.json", "w",  encoding='utf8') as file:
        json.dump(result, file, indent=4, ensure_ascii=False)

    return result

In [5]:
en_system_prompt = """You are an expert assistant in Open-Domain Question Answering. Answer the following list of questions based on your knowledge and any external materials or sources available to you, adhering to these guidelines:

**Guidelines for Answering**:
- Provide a concise, factual answer for each question using relevant information from both internal knowledge and external sources.
- Treat each question as independent, even though they are listed together.
- Each answer should strictly start with the unique ID associated with its question.
- Use only singular nouns or names in their base form, avoiding declensions and articles.
- Use no punctuation, commas, or periods.
- If multiple answers exist, separate them with a special token (e.g., "|") and sort them by relevance (the most famous one first).
- If the answer is unknown or ambiguous, make a best guess based on common or widely accepted knowledge from available sources, but avoid speculation. Use "Unknown" only if no answer can reasonably be provided from any source.

**Example**:

Questions:
1782) What is the largest planet in solar system?
78) Who starred in the movie Titanic?
9231) When did World War II start?

Answers:
1782) Jupyter
78) Leonardo DiCaprio | Kate Winslet | Billy Zane | Kathy Bates
9231) 01.09.1939
"""

In [6]:
ru_system_prompt = """Вы являетесь экспертом в области ответов на вопросы с открытой областью знаний. Ответьте на следующий список вопросов, основываясь на своих знаниях и любых доступных внешних материалах или источниках, соблюдая эти рекомендации:

Рекомендации для ответов:
- Предоставьте краткий, фактический ответ на каждый вопрос, используя релевантную информацию из внутренних знаний и внешних источников.
- Рассматривайте каждый вопрос как независимый, даже если они перечислены вместе.
- Каждый ответ должен строго начинаться с уникального идентификатора, связанного с его вопросом.
- Используйте только существительные в единственном числе или имена в их базовой форме, избегая склонений и артиклей.
- Не используйте пунктуацию, запятые или точки.
- Если существует несколько ответов, разделяйте их специальным символом (например, "|") и сортируйте по релевантности (самый известный — первый).
- Если ответ неизвестен или неоднозначен, сделайте лучший предположительный вывод, основанный на общих или широко принятых знаниях из доступных источников, но избегайте спекуляций. Используйте "Неизвестно" только в том случае, если ответ нельзя разумно предоставить из любого источника.

Пример:

Вопросы:
1782) Какая самая большая планета в Солнечной системе?
78) Кто сыграл главную роль в фильме Титаник?
9231) Когда началась Вторая мировая война?

Ответы: 
1782) Юпитер
78) Леонардо ДиКаприо | Кейт Уинслет | Билли Зейн | Кэти Бейтс
9231) 01.09.1939"""

In [44]:
DATASETS = {
    # 'rubq': get_rubq_questions(),
    # 'qald': get_qald_questions(),
    # 'lcquad': get_lcquad_questions(),
    'pat': get_pat_questions()
}

for name, dataset in DATASETS.items():
    if name == 'rubq':
        lang = 'ru'
    else:
        lang = 'en'
    task_list = create_task_list(dataset, batch_size=200, lang=lang)
    completed_tasks, failed_tasks = await client.process_tasks(task_list, verbose=True)
    assert len(failed_tasks) == 0
    result = save_answer(completed_tasks, name)

    df = pd.DataFrame(dataset)
    df['gpt'] = pd.Series(result)
    df.to_csv(f'../data/gpt_results/{name}.csv')

Лимиты: 499 запросов, 26049 токенов | Токены: 155757:  14%|▏| 1/7 [00:18<01:52, 

✅ Final success for 0


Лимиты: 499 запросов, 25441 токенов | Токены: 160585:  29%|▎| 2/7 [00:41<01:45, 

✅ Final success for 1


Лимиты: 499 запросов, 25264 токенов | Токены: 165494:  43%|▍| 3/7 [01:04<01:28, 

...

✅ Final success for 4


Лимиты: 499 запросов, 25347 токенов | Токены: 180363:  86%|▊| 6/7 [02:16<00:22, 

✅ Final success for 5


Лимиты: 499 запросов, 28092 токенов | Токены: 181403: 100%|█| 7/7 [02:18<00:00, 

✅ Final success for 6
